# Weather Data

In [ ]:
#!pip install meteostat
#!pip install pyairnow

Data Structure
Each hour is represented by a Pandas DataFrame row which provides the weather data recorded at that time. These are the different columns:

Column	Description	Type
station	The Meteostat ID of the weather station (only if query refers to multiple stations)	String
time	The datetime of the observation	Datetime64
temp	The air temperature in °C	Float64
dwpt	The dew point in °C	Float64
rhum	The relative humidity in percent (%)	Float64
prcp	The one hour precipitation total in mm	Float64
snow	The snow depth in mm	Float64
wdir	The average wind direction in degrees (°)	Float64
wspd	The average wind speed in km/h	Float64
wpgt	The peak wind gust in km/h	Float64
pres	The average sea-level air pressure in hPa	Float64
tsun	The one hour sunshine total in minutes (m)	Float64
coco	The weather condition code	Float64


In [ ]:
# Import Meteostat library and dependencies
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations, Hourly

# Get weather stations ordered by distance to Austin, TX 
#(30.4895484046493" N, 97.7249632541566" W) || Latitude of Austin (TX) 30.267153. Longitude of Austin (TX) -97.7430608.
Aus_lat = 30.267153
Aus_long = -97.7430608

# Set time period
start = datetime(2005, 1, 1)
end = datetime(2021, 12, 31, 23, 59)

#Fetch closest station (limit = 1)
stations = Stations()
stations = stations.nearby(Aus_lat, Aus_long)
stations = stations.inventory('daily', (start, end))
location = stations.fetch(1)

# Create Point for Austin, TX
#location = Point(Aus_lat, Aus_long, 70)

# Get daily data for 2005 to 2021
data = Daily(location, start, end)
#data_h = Hourly(location, start, end) 

# Plot line chart including average, minimum and maximum temperature
#data.plot(y=['tavg', 'tmin', 'tmax'])
#plt.show()

The normalize method makes sure that gaps in the time series are filled. necessary before interpolation

Aggregate Functions
Meteostat uses the following aggregate functions:

temp => mean
dwpt => mean
rhum => mean
tavg => mean
tmin => min
tmax => max
prcp => sum
snow => max
wdir => meteostat.utilities.aggregations.degree_mean
wspd => mean
wpgt => max
pres => mean
tsun => sum
coco => max

In [ ]:
data = data.normalize()
data = data.interpolate()
data_w = data.aggregate('1W') # Weekly aggregation according to above function
data_w = data_w.fetch()

data = data.aggregate('1D')
data = data.fetch()

data.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2005-01-01,21.1,17.8,23.9,0.0,0.0,NaN,11.5,NaN,1019.3,0.0
2005-01-02,20.6,18.3,22.2,8.1,0.0,NaN,11.2,NaN,1019.7,0.0
2005-01-03,22.8,20.0,25.0,1.8,0.0,NaN,13.3,NaN,1020.2,0.0
2005-01-04,22.8,19.4,25.6,0.5,0.0,NaN,15.1,NaN,1020.6,0.0
2005-01-05,12.8,3.9,21.1,2.5,0.0,NaN,14.0,NaN,1021.1,0.0


In [ ]:
print(data.shape)
print(data_w.shape)

(6209, 10)
(888, 10)


In [ ]:
data.to_csv("WeatherData_weekly.csv")
data_w.to_csv("WeatherData_weekly.csv")

In [ ]:
data_w.describe()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,888.000000,888.000000,888.000000,888.000000,888.000000,251.000000,888.000000,0.0,888.000000,888.000000
mean,21.003941,11.477815,31.517568,17.089302,0.230856,158.891235,7.953604,NaN,1016.444820,506.817568
std,7.251894,8.950844,5.850357,28.520209,5.199262,106.643329,2.093300,NaN,4.613467,1190.330275
min,-0.100000,-13.800000,10.600000,0.000000,0.000000,0.400000,2.200000,NaN,1005.700000,0.000000
25%,15.550000,3.900000,27.200000,0.000000,0.000000,65.950000,6.500000,NaN,1013.175000,0.000000
50%,21.550000,11.100000,31.700000,3.450000,0.000000,153.500000,7.900000,NaN,1015.500000,0.000000
75%,27.400000,20.600000,36.100000,23.725000,0.000000,209.300000,9.300000,NaN,1019.400000,0.000000
max,33.300000,25.600000,44.400000,201.600000,150.000000,359.100000,16.200000,NaN,1033.700000,5069.000000


In [ ]:
data.describe()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,1542.000000,6209.000000,0.0,6179.000000,6209.000000
mean,21.005637,15.452070,27.520551,2.444081,0.102271,163.520817,7.952311,NaN,1016.457000,72.484136
std,7.807130,8.063366,8.357006,9.461687,3.257131,109.044366,3.534772,NaN,5.891307,189.689991
min,-9.200000,-13.800000,-3.800000,0.000000,0.000000,0.000000,0.400000,NaN,998.300000,0.000000
25%,15.400000,8.900000,22.200000,0.000000,0.000000,73.250000,5.400000,NaN,1012.500000,0.000000
50%,22.200000,17.200000,28.300000,0.000000,0.000000,161.900000,7.600000,NaN,1015.300000,0.000000
75%,27.600000,22.800000,34.400000,0.000000,0.000000,211.000000,10.400000,NaN,1019.700000,0.000000
max,35.500000,28.900000,44.400000,178.800000,150.000000,359.000000,25.900000,NaN,1045.300000,841.000000


# Pollution data

In [ ]:
'''import requests 
from six.moves import urllib

APIkey = 'C0944E2A-D5D0-4231-ABAB-5F7530450F2A'

#&zipCode=78708&date=2022-03-18T00-0000&distance=50&API_KEY=C0944E2A-D5D0-4231-ABAB-5F7530450F2A
url = 'https://www.airnowapi.org/aq/observation/latLong/historical/?format=text/csv&latitude={}&longitude={}&date={}T00-0000&distance=50&API_KEY={}'.format(Aus_lat,Aus_long,start,APIkey)

Pdata = requests.get(url)
airdata = Pdata.json()
print(airdata)
'''


"import requests \nfrom six.moves import urllib\n\nAPIkey = 'C0944E2A-D5D0-4231-ABAB-5F7530450F2A'\n\n#&zipCode=78708&date=2022-03-18T00-0000&distance=50&API_KEY=C0944E2A-D5D0-4231-ABAB-5F7530450F2A\nurl = 'https://www.airnowapi.org/aq/observation/latLong/historical/?format=text/csv&latitude={}&longitude={}&date={}T00-0000&distance=50&API_KEY={}'.format(Aus_lat,Aus_long,start,APIkey)\n\nPdata = requests.get(url)\nairdata = Pdata.json()\nprint(airdata)\n"

In [ ]:
import pandas as pd 

PMdata = pd.read_csv('./austin-texas-air-quality.csv')
PMdata.head()

,date,pm25,o3,no2,so2
0,2022/3/1,45,,,
1,2022/3/2,49,,,
2,2022/3/3,39,,,
3,2022/3/4,33,,,
4,2022/3/5,33,,,


In [ ]:
PMdata.tail()

,time,pm25,o3,no2,so2
2695,2014-03-29,21,,,
2696,2014-03-30,29,,,
2697,2014-03-31,35,,,
2698,2016-10-29,,26,,
2699,2016-10-30,,14,,


In [ ]:
PMdata['date'] = pd.to_datetime(PMdata["date"].astype(str), format='%Y-%m-%d')
PMdata.rename(columns = {'date':'time'}, inplace = True)
PMdata.set_index('time')
print(PMdata.shape)
PMdata.head()

(2700, 5)


,time,pm25,o3,no2,so2
0,2022-03-01,45,,,
1,2022-03-02,49,,,
2,2022-03-03,39,,,
3,2022-03-04,33,,,
4,2022-03-05,33,,,


In [ ]:
Weather_merged_daily = pd.merge(data, PMdata, on=['time'])
Weather_merged_daily.to_csv('Weather_merged_daily.csv')
print(Weather_merged_daily.shape)
Weather_merged_daily.head()

(2624, 15)


,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,pm25,o3,no2,so2
0,2014-01-01,10.9,3.9,20.6,0.0,0.0,NaN,6.1,NaN,1020.2,0.0,71,,,
1,2014-01-02,6.8,0.6,12.8,0.0,0.0,NaN,12.2,NaN,1024.2,0.0,17,,,
2,2014-01-03,5.6,-2.1,14.4,0.0,0.0,NaN,6.8,NaN,1028.2,0.0,26,,,
3,2014-01-04,13.8,5.6,21.7,0.0,0.0,NaN,10.4,NaN,1014.6,0.0,26,,,
4,2014-01-05,8.2,1.1,15.0,0.0,0.0,NaN,15.5,NaN,1026.6,0.0,10,,,


In [ ]:
PMdata.describe()

,time,pm25,o3,no2,so2
count,2700,2700,2700,2700,2700
unique,2700,88,11,6,1
top,2022-03-01 00:00:00,33,,,
freq,1,81,2687,2694,2700
first,2014-01-01 00:00:00,NaN,NaN,NaN,NaN
last,2022-03-19 00:00:00,NaN,NaN,NaN,NaN


In [ ]:
import datetime as dt

PMdata_w = PMdata.copy()

PMdata_w['WeekDate'] = PMdata_w.apply(lambda row: row['time'] - dt.timedelta(days=row['time'].weekday()+1),axis=1)
PMdata_w[' pm25']= pd.to_numeric(PMdata_w[' pm25'],errors = 'coerce',downcast="integer")
PMdata_w[' o3']= pd.to_numeric(PMdata_w[' o3'],errors = 'coerce',downcast="integer")
PMdata_w[' no2']= pd.to_numeric(PMdata_w[' no2'],errors = 'coerce',downcast="integer")
PMdata_w[' so2']= pd.to_numeric(PMdata_w[' so2'],errors = 'coerce',downcast="integer")
#print(PMdata_w.dtypes)
perweek = PMdata_w[' pm25'].groupby(PMdata_w['WeekDate']).mean()
perweek

WeekDate
2013-12-29    30.000000
2014-01-05    26.000000
2014-01-12    29.571429
2014-01-19    28.714286
2014-01-26    23.142857
                ...    
2022-02-13    30.857143
2022-02-20    34.857143
2022-02-27    40.428571
2022-03-06    32.857143
2022-03-13    34.500000
Name:  pm25, Length: 398, dtype: float64

In [ ]:
#perweek.rename(columns = {'WeekDate':'time'}, inplace = True)
perweek_df = pd.DataFrame(perweek)
#perweek_df = perweek_df.rename(columns = {'WeekDate':'time'})
perweek_df.index.names = ['time']
perweek_df

,pm25
time,
2013-12-29,30.000000
2014-01-05,26.000000
2014-01-12,29.571429
2014-01-19,28.714286
2014-01-26,23.142857
...,...
2022-02-13,30.857143
2022-02-20,34.857143
2022-02-27,40.428571


In [ ]:
Weather_merged_weekly = pd.merge(data_w, perweek_df, on=['time'])
print(Weather_merged_weekly.shape)
Weather_merged_weekly.head()

(388, 11)


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,pm25
time,,,,,,,,,,,
2013-12-29,8.9,-0.5,17.8,0.0,0.0,NaN,4.7,NaN,1026.9,0.0,30.000000
2014-01-05,8.0,-2.1,21.7,0.0,0.0,NaN,8.7,NaN,1023.5,0.0,26.000000
2014-01-12,10.5,-5.5,23.9,6.9,0.0,NaN,6.6,NaN,1027.3,0.0,29.571429
2014-01-19,13.0,2.8,24.4,0.0,0.0,NaN,7.0,NaN,1022.2,0.0,28.714286
2014-01-26,9.9,-3.2,27.8,2.8,0.0,NaN,7.7,NaN,1023.2,0.0,23.142857


In [ ]:
Weather_merged_weekly.to_csv('Weather_merged_weekly.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1a1cabe9-69f7-4a67-be18-e232e1feb2b6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>